In [2]:
# Some libraries
from scipy import *
from math import *
from matplotlib.pyplot import *
from functools import *
import sys
from tqdm import tqdm
import time

In [3]:
from sympy.geometry import Point
from sympy.geometry import Polygon as SimpyPolygon
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import pyclipper
import functools

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Class and converting functions

In [5]:
class Rectangle:
    """Minimal representation of a rectangle"""
    def __init__(self, point1_x, point1_y, point2_x, point2_y, longueur):
        self.point1_x = point1_x
        self.point1_y = point1_y
        self.point2_x = point2_x
        self.point2_y = point2_y
        self.longueur = longueur
        
    def side_length(self):
        return dist((self.point1_x, self.point1_y), (self.point2_x, self.point2_y))
    
    def area(self):
        return abs(self.side_length() * self.longueur)
    
    def to_rect(self, strategy="Default"):
        """Convert a minimal rectangle to a full rectangle"""
        vect = (self.point2_x - self.point1_x, self.point2_y - self.point1_y)
        distance = self.side_length()
        vectUnitaire = (vect[0] / distance * self.longueur, vect[1] / distance * self.longueur)
        normalVect = (-vectUnitaire[1], vectUnitaire[0])
        point1 = (self.point1_x, self.point1_y)
        point2 = (self.point2_x, self.point2_y)
        point3 = (self.point2_x + normalVect[0], self.point2_y + normalVect[1])
        point4 = (self.point1_x + normalVect[0], self.point1_y + normalVect[1])
        if strategy == "Default":
            return [point1, point2, point3, point4]
        else:
            return SimpyPolygon(point1, point2, point3, point4)
   
    def is_valid(self, polygon, verbose=False):
        """ Determine if the rectangle is in the polygon

        @param polygon: whether a polygon or a list of vertices
        """
        if verbose:
            print("entrée: est_valide")
            start_time = time.time()
        try:
            rectangle = self.to_rect()
            pc = pyclipper.Pyclipper()
            pc.AddPath(polygon, pyclipper.PT_CLIP, True)
            pc.AddPath(rectangle, pyclipper.PT_SUBJECT, True)

            intersections = pc.Execute(pyclipper.CT_INTERSECTION, pyclipper.PFT_EVENODD, pyclipper.PFT_EVENODD)

            res = (intersections != []) and (len(intersections[0]) == len(rectangle)) and np.all([vertice_in_polygon(v, intersections[0]) for v in rectangle]) 
            if verbose:
                print("--- %s seconds ---" % (time.time() - start_time))
            return res
        except pyclipper.ClipperException:
            print("Clipper exception")
            return False

In [6]:
def vertice_in_polygon(vertice, polygon):
    """ Determine if the vertice is in the polygon
    Not a strict check because of the pyclipper library in which
    the coordinates are floored.
    """
    x, y = vertice
    x, y = int(x), int(y)
    return [x,y] in polygon

In [7]:
def isValid_Nico(polygon, sol):
    x1, y1, x2, y2, length = sol
    SIZE=1
    if(dist((x1, y1), (x2, y2)) >= SIZE and abs(length) >= SIZE):
        rectangle = sol2rect(sol)
        
        # test intersection
        pc = pyclipper.Pyclipper()
        pc.AddPath(polygon, pyclipper.PT_CLIP, True)
        try:
            pc.AddPath(rectangle, pyclipper.PT_SUBJECT, True)
        except:
            print("rectangle invalide ", rectangle)
        intersect = pc.Execute(pyclipper.CT_INTERSECTION, pyclipper.PFT_EVENODD, pyclipper.PFT_EVENODD)
        
        if len(intersect) > 0:
            return sorted(intersect[0]) == sorted([[int(x), int(y)] for [x,y] in rectangle])
    
    return False

## Drawing functions

In [8]:
def draw_polygon(polygon, ax=None, color="green"):
    """Draw a polygon over an axis (not shown directly)
    
    @param polygon: Whether a polygon or a path
    """
    if ax is None:
        ax = plt.gca()
    if type(polygon) == SimpyPolygon:
        polygon = np.array(list(map(lambda p: [p.x, p.y], polygon.vertices)))
    p = Polygon(polygon, closed=False, color=color, alpha=0.3, lw=0)
    ax.add_patch(p)
    ax.axis('equal')
    return ax

In [9]:
def draw_polygons(polygons, colors=None, verbose=False):
    """Draw polygons and print the figure
    
    @param polygons: SimpyPolygon or list(SimpyPolygon)
    @param colors: colors to match to the polygons
    """
    ax = plt.gca()
    if type(polygons) == SimpyPolygon:
        polygons = [polygons]
    if colors is None:
        colors = ["grey"]*len(polygons)
    if len(colors) > 0 and len(colors) < len(polygons):
        colors = colors + [colors[-1]] * (len(polygons) - len(colors))
    colors.reverse()
    
    for polygon in polygons:
        color = colors.pop()
        if verbose:
            print(color, "Polygon : ", polygon)
        ax = draw_polygon(polygon, ax, color=color)
    plt.show()

In [10]:
def log_polygons(land, sols, colors=["green", "brown"], legend="Best", padding=1, iteration=0):
    if (iteration % padding == 0):
        sols = list(sols) if type(sols) == map else [sols]
        legend_text = " : " + legend if legend is not None else ""
        printmd("### Itération "+str(iteration)+legend_text)
        polygons = [land]+[Rectangle(*sol).to_rect() for sol in sols]
        draw_polygons(polygons, colors)

## Calculate the area of a polygon

In [11]:
def get_bornage_search_field(polygon):
    polygon = polygon_to_path(polygon)
    min_func = lambda acc, vertice: min(min(*vertice),acc)
    max_func = lambda acc, vertice: max(max(*vertice),acc)
    
    min_search_field = functools.reduce(min_func, polygon, sys.maxsize)
    max_search_field = functools.reduce(max_func, polygon, -sys.maxsize)
    
    return min_search_field, max_search_field

## Calculate the validity of a solution

In [12]:
def polygon_bornage(sol, land_polygon, verbose=False):
    if verbose:
        print("entrée: polygon_bornage")
    solution = Rectangle(*sol)
    if solution.is_valid(land_polygon):
        if verbose:
            print(sol)
            printmd("## Est valide")
        return sol 
    else:
        if verbose:
            print("this one is invalid")
        # print(sol)
        raise ValueError("Solution invalide par contrainte de bornage")

In [13]:
Nb_cycles = 2000
Nb_particle = 20
# usual params
psi,cmax = (0.4, 1.12)
# psi,cmax = (0,7, 1,47)
# psi,cmax = (0.8, 1.62)

### Calcul de temps

In [14]:
from sympy import Symbol
def pick_random_sol(land, size=1, delta=0.05, stochastic=False):
    t = Symbol('t', real=True)
    p = SimpyPolygon(*land).arbitrary_point('t')
    if stochastic:
        picks = sorted([random.rand() for i in range(size*2)])
    else:
        picks = np.linspace(0, 1, size*2+1)
    points_on_perimeter = [p.subs(t, pick) for pick in picks]
    #picks = [pick + delta if pick + delta < 1 else pick + (1 - pick) / 2 for pick in picks]
    #second_points_on_perimeter = [p.subs(t, pick) for pick in picks]
    sols = [[points_on_perimeter[i*2].x, points_on_perimeter[i*2].y, points_on_perimeter[i*2+1].x, points_on_perimeter[i*2+1].y, -10-delta*2000*random.rand()] for i in range(size)]
    for sol in sols:
        while not Rectangle(*sol).is_valid(land):
            sol[4] = sol[4] / 2
    return sols
    # return p.subs(t, (s1 + s2/2)/perimeter)
# sols = [Rectangle(*sol) for sol in pick_random_sol(land, 50)]
# rects = [s.to_rect() for s in sols]
# draw_polygons([*rects, land], colors=["red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red", "green"])
# print("Tous valides ? ", list(all(sol.is_valid(land) for sol in sols)))

In [15]:
# Init of the population (swarm)
def initSwarm(nb,dim,eval_func,land=None):
    positions = pick_random_sol(land, nb)
    fits = [eval_func(pos) for pos in positions]
    return [{'vit':[0]*dim, 'pos':positions[i], 'fit':fits[i], 'bestpos':positions[i], 'bestfit':fits[i], 'bestvois':[]} for i in range(nb)]

In [16]:
# Return the particle with the best fitness
def maxParticle(p1,p2):
    if (p1["fit"] > p2["fit"]):
        return p1 
    else:
        return p2

# Returns a copy of the particle with the best fitness in the population
def getBest(swarm):
    return dict(reduce(lambda acc, e: maxParticle(acc,e),swarm[1:],swarm[0]))

In [17]:
# Update information for the particles of the population (swarm)
def update(particle,bestParticle):
    nv = dict(particle)
    if(particle["fit"] > particle["bestfit"]):
        nv['bestpos'] = particle["pos"][:]
        nv['bestfit'] = particle["fit"]
    nv['bestvois'] = bestParticle["bestpos"][:]
    return nv

# Calculate the velocity and move a particule
def move(particle, dim, eval_function, bornage_function, psi, cmax, verbose=False):
    if verbose:
        print("entrée: move")
    nv = dict(particle)

    velocity = [0]*dim
    for i in range(dim):
        velocity[i] = (particle["vit"][i]*psi + \
        cmax*random.uniform()*(particle["bestpos"][i] - particle["pos"][i]) + \
        cmax*random.uniform()*(particle["bestvois"][i] - particle["pos"][i]))

    new_pos = [particle["pos"][i] + velocity[i] for i in range(dim)]
    
    start_time = time.time()
    try:
        position = bornage_function(new_pos)   
    except ValueError:
        position = particle["pos"]
    if verbose:
        print("--- Bornage: %s seconds ---" % (time.time() - start_time))
    
    nv['vit'] = velocity
    nv['pos'] = position
    nv['fit'] = eval_function(position)
    if verbose:
        print("Debug: ", particle["pos"], "is the previous solution")
        print("Debug: ", new_pos, "is the new solution")
        print("sortie: move")
    return nv

In [20]:
def sol_to_rect(sol):
    """Convert a minimal rectangle to a full rectangle"""
    x1, x2, y1, y2, length = sol
    vect = (x2 - x1, y2 - y1)
    distance = dist((x1,y1),(x2,y2))
    vectUnitaire = (vect[0] / distance * length, vect[1] / distance * length)
    normalVect = (-vectUnitaire[1], vectUnitaire[0])
    point1 = (x1, y1)
    point2 = (x2, y2)
    point3 = (x2 + normalVect[0], y2 + normalVect[1])
    point4 = (x1 + normalVect[0], y1 + normalVect[1])
    return [point1, point2, point3, point4]

def estValide(rectangle,polygon):
    try:
        pc = pyclipper.Pyclipper()
        pc.AddPath(polygon, pyclipper.PT_SUBJECT, True)
        pc.AddPath(rectangle, pyclipper.PT_CLIP, True)
    
        clip = pc.Execute(pyclipper.CT_INTERSECTION, pyclipper.PFT_EVENODD, pyclipper.PFT_EVENODD)
        return (clip!= []) and (len(clip[0]) == len(rectangle)) and np.all([[int(coord) for coord in sommet] in clip[0] for sommet in rectangle]) 
    except pyclipper.ClipperException:
        return False
    
def sol2rect(solution):
    x1, y1, x2, y2, length = solution
    normal_vect = [-(y2-y1), (x2-x1)]
    
    coeff = length / hypot(normal_vect[0], normal_vect[1])
    x3, y3 = x2 + normal_vect[0] * coeff, y2 + normal_vect[1] * coeff
    x4, y4 = x1 + normal_vect[0] * coeff, y1 + normal_vect[1] * coeff
    return [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]

In [22]:
dim = 5
eval_function = lambda x: Rectangle(*x).area()
bornage_function = lambda x: polygon_bornage(x, land)
log_function = lambda x,legend=None,padding=100,colors=["green", "red"]:log_polygons(land, x, colors, legend, padding)
land = [(0,0), (0,100), (100,100), (110, 50), (100, 0)]

print("initSwarm")
%time swarm = initSwarm(20,dim,eval_func=eval_function, land=land)

best = getBest(swarm)
%time swarm = [update(e,best) for e in swarm]

print("move")
%time swarm = [move(particle=e, dim=dim, eval_function=eval_function, bornage_function=bornage_function, psi=0.7, cmax=1.47) for e in swarm]

print("bounding")
%time [bornage_function(e["pos"]) for e in swarm]

print("moi")
%time sols = [Rectangle(*e["pos"]) for e in swarm]
%time [s.is_valid(land) for s in sols]

print("Nico")
%time [isValid_Nico(land, e["pos"]) for e in swarm]

print("ju")
%time sols = [sol_to_rect(e["pos"]) for e in swarm]
%time [estValide(s, land) for s in sols]


initSwarm
CPU times: user 3.63 s, sys: 3.77 ms, total: 3.63 s
Wall time: 3.63 s
CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 17.9 µs
move
CPU times: user 2.99 s, sys: 0 ns, total: 2.99 s
Wall time: 2.99 s
bounding
CPU times: user 1.43 s, sys: 3.81 ms, total: 1.44 s
Wall time: 1.44 s
moi
CPU times: user 216 µs, sys: 2 µs, total: 218 µs
Wall time: 221 µs
CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.44 s
Nico
CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s
ju
CPU times: user 220 ms, sys: 0 ns, total: 220 ms
Wall time: 219 ms
CPU times: user 753 ms, sys: 0 ns, total: 753 ms
Wall time: 752 ms


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [ ]:
def fit(eval_function, 
        bornage_function, 
        nb_particle=Nb_particle, 
        dim=5, 
        nb_cycles=Nb_cycles, 
        psi=0.7,
        cmax=1.47,
        log_function=print):
    Htemps = []       # temps
    Hbest = []        # distance

    # initialization of the population
    swarm = initSwarm(nb_particle,dim,eval_func=eval_function, land=LAND)
    log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="Initialization", padding=1)
    # initialization of the best solution
    best = getBest(swarm)
    best_cycle = best

    for i in tqdm(range(nb_cycles)):
        #Update informations
        swarm = [update(e,best_cycle) for e in swarm]
        # velocity calculations and displacement
        swarm = [move(particle=e, dim=dim, psi=psi, cmax=cmax, eval_function=eval_function, bornage_function=bornage_function) for e in swarm]
        log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="Swarm", it=i+1)
        # Update of the best solution
        best_cycle = getBest(swarm)
        if (best_cycle["bestfit"] > best["bestfit"]):
            best = best_cycle
            # draw(best['pos'], best['fit'])

        # historization of data
        if i % 10 == 0:
            Htemps.append(i)
            Hbest.append(best['bestfit'])
            
        log_function(best['pos'], legend=f"Best = **{int(best['bestfit'])}**", colors=["green", "blue"], it=i+1)

    # END, displaying results
    Htemps.append(i)
    Hbest.append(best['bestfit'])

    #displaying result on the console
    log_function(best['pos'], legend=f"Best final = **{int(best['bestfit'])}**", colors=["green", "blue"], it=nb_cycles)
    print(Htemps, Hbest)

In [ ]:
fit(eval_function=lambda x: Rectangle(*x).area(), 
    bornage_function=lambda x: polygon_bornage(x, land), 
    log_function=lambda x,it=0,legend=None,padding=10,colors=["green", "red"]:log_polygons(land, x, colors, legend, padding, it),
    nb_particle=3, 
    dim=5, 
    nb_cycles=50,
    psi=0.7,
    cmax=1.47)

In [ ]:
sol = {'vit': [-2.52650027135325e-9 - 9.500520874982e-13*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) - 9.03388401872037e-13*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) - 6.56247635134616e-13*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 7/8) - 5.83755580724892e-13*(1/4 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 9.23325889916609e-13*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) + 1.42751642707792e-9*(3/8 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3), -1.60391696626868e-11*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 5.44237596800181e-10*sqrt(26) + 6.23355197126566e-12*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 1.87050362530917e-11*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 1.0597633262519e-8, -7.38902365445221e-9*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 13/16) - 5.30958966523229e-12*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) - 2.57285545709561e-12*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16) + 7.85333125056432e-13*(5/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 7.65097188497398e-11*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 15/16) + 8.65938250446284e-10 + 2.49607482420176e-9*(7/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3), -3.16662145494745e-7 + 4.87280383241628e-9*sqrt(26) + 3.63478088955149e-10*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16) + 6.17773260361117e-10*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300), -2.2218150881391868e-08], 'pos': [-0.0241654708414921*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 8.5611058925468*(3/8 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.00043537198601596*(1/4 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.000420581098437307*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 7/8) + 0.0316976964042511*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 0.0501606196808244*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 108.875025479554, -0.0684100108128167*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 0.497773125420269*sqrt(26) - 0.278131935063796*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.106687467083637*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 122.224453106317, -19.0876818950928*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 13/16) - 0.00405072542958905*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16) - 2.13831496808409e-5*(5/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.00723590339811345*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 15/16) + 2.91246375812911*(7/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.0262480695514679*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 98.9105015541361, -11.031235779703 - 0.0364900857505641*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.341525397560728*sqrt(26) + 0.14937055540896*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16), -99.60562828197416], 'fit': 9525.36690839256, 'bestpos': [-0.0241654708424154*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 8.56110589397431*(3/8 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.000435371986599716*(1/4 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.000420581099093555*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 7/8) + 0.0316976964051545*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 0.0501606196817745*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 108.875025482081, -0.0684100107967775*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 0.497773124876032*sqrt(26) - 0.278131935082501*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.106687467077403*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 122.22445309572, -19.0876818877038*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 13/16) - 0.00405072542701619*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16) - 2.1383150466174e-5*(5/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.00723590332160373*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 15/16) + 2.91246375563303*(7/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.0262480695567775*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 98.9105015532701, -11.0312354630408 - 0.0364900863683373*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.341525392687925*sqrt(26) + 0.149370555045481*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16), -99.60562825975602], 'bestfit': 9525.366889699919, 'bestvois': [-0.0241654708410253*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 8.56110589182525*(3/8 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.000435371985717388*(1/4 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.000420581098102114*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 7/8) + 0.0316976964037945*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 0.0501606196803444*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 108.875025478277, -0.0684100108864031*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 3/4) - 0.497773127917988*sqrt(26) - 0.278131934978649*sqrt(26)*(1/2 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.106687467112496*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 5/8) + 122.224453155028, -19.0876819025199*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 13/16) - 0.00405072543217502*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16) - 2.13831488897166e-5*(5/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.00723590347551493*(sqrt(26)/5 + 3)*(-200/(20*sqrt(26) + 300) - 20*sqrt(26)/(20*sqrt(26) + 300) + 15/16) + 2.91246376063626*(7/16 - 100/(20*sqrt(26) + 300))*(sqrt(26)/5 + 3) + 0.0262480695461357*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 98.9105015550178, -11.0312348641445 - 0.0364900875367143*sqrt(26)*(9/16 - 200/(20*sqrt(26) + 300))*(20*sqrt(26) + 300) + 0.341525383472843*sqrt(26) + 0.149370554358046*sqrt(26)*(20*sqrt(26) + 300)*(-200/(20*sqrt(26) + 300) - 10*sqrt(26)/(20*sqrt(26) + 300) + 11/16), -99.60562831151383]}
draw_polygons([land, Rectangle(*sol["pos"]).to_rect()], colors=["green", "blue"])
sol["pos"]

In [ ]:
sol = [98.60236160251716,
 100.80276931888929,
 99.395493294684,
 5.175248415058288,
 -99.60562828197416]
st = time.time()
rectangle=sol_to_rect(sol)
estValide(rectangle, land)
print(time.time() - st)
st = time.time()
Rectangle(*sol).is_valid(land)
print(time.time() - st)